# Simple SDMX > V3 loadfile transformation

Its "simple" as we aren't using the urn's etc, just using a provided codelist (sdmx_ref.xlsx) for transforming the data.

Longer term we'll need to leverage the global SDMX resources.

This code block just constructs some lookups (a python dictionary) we can use to translate the SDMX codes to readable text later. There's a preview of sorts at the end of this block showing what's in a typical single 'lookup'.


In [195]:

# Get all the SDMX codes into a dictionary we can use called "lookups"

from databaker.framework import *
from pprint import pprint

tabs = loadxlstabs('sdmx_ref.xlsx')

lookups = {}
for tab in tabs:
    
    if tab.name != 'Overview' and tab.name != 'Matrix':
        
        colA = tab.excel_ref('A2').expand(DOWN).is_not_blank()
        
        aTuple = ()
        for ob in colA:
            aTuple = aTuple + (ob.value,)
            
        colB = tab.excel_ref('B2').expand(DOWN).is_not_blank()
            
        bTuple = ()
        for ob in colB:
            bTuple = bTuple + (ob.value,)
            
        
        assert len(aTuple) == len(bTuple)
        
        oneDict = {}
        for i in range(0, len(aTuple)):
            oneDict.update({aTuple[i]:bTuple[i]})
            
        lookups.update({tab.name[3:]:oneDict})

        
# preview of sorts
tabs = [x for x in tabs if x.name != 'Overview' and x.name != 'Matrix']

print('\n\nExample: This is a single "Lookup":\n')
pprint(lookups[tabs[0].name[3:]])
print('\nThere will be one of these generated for each of the Table names above')
        
        

Loading sdmx_ref.xlsx which has size 608254 bytes
Table names: ['Overview', 'Matrix', 'CL_FREQ', 'CL_AREA', 'CL_SEASONAL_ADJUST', 'CL_STS_INDICATOR', 'CL_ACTIVITY_STS', 'CL_BASE_PER', 'CL_OBS_STATUS', 'CL_CONF_STATUS', 'CL_UNIT_MULT', 'CL_UNIT', 'CL_DECIMALS', 'CL_TRANSFORMATION', 'CL_TIME_FORMAT']


Example: This is a single "Lookup":

{'A': 'Annual',
 'B': 'Daily ? businessweek',
 'D': 'Daily',
 'H': 'Hourly',
 'M': 'Monthly',
 'N': 'Minutely',
 'Q': 'Quarterly',
 'S': 'Half-yearly, semester',
 'W': 'Weekly'}

There will be one of these generated for each of the Table names above


## Flatten the SMDX

Literally that. No other changes, 10 line preview at the end.

In [196]:
# flatten the sdmx

inputfile = 'T0111Quarterly_4(SDMXExport) (11).xml'

from bs4 import BeautifulSoup
import pandas as pd

with open(inputfile, 'r') as f:
    soup = BeautifulSoup(f, 'lxml')

    # for each data series
    dataSeries = soup.html.body.compactdata.find_all('na_:series')
    
    # build dict
    finalDict = {}
    initial_keys = ['obs_value', 'time_period', 'obs_status', 'conf_status']
    for ik in initial_keys:
        finalDict.update({ik:[]})
    
    # add the keys is use per series
    series_keys = dataSeries[0].attrs.keys()
    for sk in series_keys:
        finalDict.update({sk:[]})
        
        
    # EXTRACT
    for dSeries in dataSeries:
        for ob in dSeries.findChildren():    
            for ik in initial_keys:
                finalDict[ik].append(ob[ik])
            for sk in series_keys:
                finalDict[sk].append(dSeries[sk])
                
    obs_file = pd.DataFrame.from_dict(finalDict)


obs_file[:10] # preview


,accounting_entry,activity,adjustment,comment_ts,compiling_org,conf_status,counterpart_area,counterpart_sector,decimals,expenditure,...,prices,ref_area,ref_sector,sto,table_identifier,time_period,title,transformation,unit_measure,unit_mult
0,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1995-Q1,Employment by Industry,N,HW,6
1,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1995-Q2,Employment by Industry,N,HW,6
2,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1995-Q3,Employment by Industry,N,HW,6
3,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1995-Q4,Employment by Industry,N,HW,6
4,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1996-Q1,Employment by Industry,N,HW,6
5,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1996-Q2,Employment by Industry,N,HW,6
6,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1996-Q3,Employment by Industry,N,HW,6
7,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1996-Q4,Employment by Industry,N,HW,6
8,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1997-Q1,Employment by Industry,N,HW,6
9,_Z,A,N,\n,GB1,F,W2,S1,3,_Z,...,_Z,GB,S1,EMP,T0111,1997-Q2,Employment by Industry,N,HW,6


# Tidy things up a bit

change order to something more like we're used to. switch the in-file headings to match their equivilents in the lookup dict.

10 line preview at the end.


In [197]:
# order it to be more like our typcal load files

reorder = ['obs_value',
'obs_status',
'time_period',
'ref_area',        
'unit_measure',
'unit_mult',
'title',
'accounting_entry',
'activity',
'adjustment',
'comment_ts',
'compiling_org',
'conf_status',
'counterpart_area',
'counterpart_sector',
'decimals',
'expenditure',
'freq',
'instr_asset',
'prices',
'ref_sector',
'sto',
'table_identifier',
'transformation']

obs_file = obs_file[reorder]


# we also need to update some of the in-file headings to match those used in the lookup xls
headerChange = {
    'ref_area':'area',
    'adjustment':'seasonal_adjustment',
    'unit_measure':'unit',
    'activity':'activity_sts'
}
obs_file = obs_file.rename(columns=headerChange)

obs_file[:10] # preview


,obs_value,obs_status,time_period,area,unit,unit_mult,title,accounting_entry,activity_sts,seasonal_adjustment,...,counterpart_sector,decimals,expenditure,freq,instr_asset,prices,ref_sector,sto,table_identifier,transformation
0,279.902,A,1995-Q1,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
1,303.099,A,1995-Q2,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
2,307.714,A,1995-Q3,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
3,285.639,A,1995-Q4,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
4,258.886,A,1996-Q1,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
5,291.308,A,1996-Q2,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
6,302.456,A,1996-Q3,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
7,277.518,A,1996-Q4,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
8,255.679,A,1997-Q1,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N
9,291.108,A,1997-Q2,GB,HW,6,Employment by Industry,_Z,A,N,...,S1,3,_Z,Q,_Z,_Z,S1,EMP,T0111,N


## Perform Lookups

get rid of some of those SDMX codes, replace with plain text.

10 line preview at the end.

In [198]:
# lookup the values ... do we even want to do this? debatable. SDMX is more fully ...
# ... described than text and that should be a good thing.

for i, row in obs_file.iterrows():
    
    for header in obs_file.columns.values:
        
        try:
            obs_file.ix[i, header] = lookups[header.upper()][obs_file.ix[i, header]]
        except:
            pass
        
        if obs_file.ix[i, header] == '_Z':  # dump "non applicable entries"
            obs_file.ix[i, header] = ''


# hacky "drop empty columns" code.

# if a columns has one unique entry equal to '' add to list to drop
dropList = []
for col in obs_file.columns.values:
        if len(obs_file[col].unique()) == 1 and obs_file[col].unique()[0] == '':
            dropList.append(col)

# then drop
for drop in dropList:
    
    # get column index number, then drop the column
    delMe = obs_file.columns.get_loc(drop)
    obs_file = obs_file.drop(obs_file.columns[delMe], axis=1)

obs_file[:10]  # preview


,obs_value,obs_status,time_period,area,unit,unit_mult,title,activity_sts,seasonal_adjustment,comment_ts,compiling_org,conf_status,counterpart_area,counterpart_sector,decimals,freq,ref_sector,sto,table_identifier,transformation
0,279.902,Normal value,1995-Q1,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
1,303.099,Normal value,1995-Q2,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
2,307.714,Normal value,1995-Q3,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
3,285.639,Normal value,1995-Q4,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
4,258.886,Normal value,1996-Q1,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
5,291.308,Normal value,1996-Q2,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
6,302.456,Normal value,1996-Q3,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
7,277.518,Normal value,1996-Q4,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
8,255.679,Normal value,1997-Q1,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data
9,291.108,Normal value,1997-Q2,United Kingdom,Hours worked,6,Employment by Industry,"Agriculture, forestry and fishing",N,\n,GB1,Free (free for publication),W2,S1,3,Quarterly,S1,EMP,T0111,Non transformed data


## Creating a V3 Version

This mainly consists of disregarding columns that aren't relevant to our format.

Interestingly, a lot of these (unit of measure, title etc) are metadata and could feasibly be piped in that direction instead.

Disregarded columns containing (arguably) metadata:

* title
* unit
* unit multiplier - SELF: INCORPORATE THIS
* compiling org
* frequency

10 line preview at the end.


In [199]:

# anything that we dont want generating its own dimension
# time and geo and anything relating to obs are handled specially so make the list

# dumping as pointless
disregard = ['obs_value', 'obs_status', 'time_period', 'area', 'unit', 'unit_mult', 'table_identifier', 
             'transformation', 'freq', 'comment_ts', 'decimals', 'conf_status', 'compiling_org', 'title']
    
v3_file = pd.DataFrame(columns=['Observation', 'Data_Marking', 'Observation_Type_Value'])
v3_file['Observation'] = obs_file['obs_value']

# dim 1: time
v3_file['Dimension_Hierarchy_1'] = 'time'
v3_file['Dimension_Name_1'] = ''
v3_file['Dimension_Value_1'] = obs_file['time_period']

# dim 2: geography
v3_file['Dimension_Hierarchy_2'] = ''
v3_file['Dimension_Name_2'] = 'Geography'
v3_file['Dimension_Value_2'] = obs_file['area']

# whatevers left
repeatHeaders = ['Dimension_Hierarchy_', 'Dimension_Name_', 'Dimension_Value_']
counter = 3 # as 1 and 2 are taken by geography
for header in obs_file.columns.values:
    if header not in disregard:
        v3_file['Dimension_Hierarchy_' + str(counter)] = ''
        v3_file['Dimension_Name_' + str(counter)] = header
        v3_file['Dimension_Value_' + str(counter)] = obs_file[header]
        counter += 1

v3_file.fillna('', inplace=True)

v3_file[:10]

,Observation,Data_Marking,Observation_Type_Value,Dimension_Hierarchy_1,Dimension_Name_1,Dimension_Value_1,Dimension_Hierarchy_2,Dimension_Name_2,Dimension_Value_2,Dimension_Hierarchy_3,...,Dimension_Value_5,Dimension_Hierarchy_6,Dimension_Name_6,Dimension_Value_6,Dimension_Hierarchy_7,Dimension_Name_7,Dimension_Value_7,Dimension_Hierarchy_8,Dimension_Name_8,Dimension_Value_8
0,279.902,,,time,,1995-Q1,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
1,303.099,,,time,,1995-Q2,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
2,307.714,,,time,,1995-Q3,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
3,285.639,,,time,,1995-Q4,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
4,258.886,,,time,,1996-Q1,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
5,291.308,,,time,,1996-Q2,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
6,302.456,,,time,,1996-Q3,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
7,277.518,,,time,,1996-Q4,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
8,255.679,,,time,,1997-Q1,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
9,291.108,,,time,,1997-Q2,,Geography,United Kingdom,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP


## Reformat time and geography for our systems

Doing these in their own section as theyre a pretty big deal.

Preview at the end.


# Custom Tidying Up

Based on the specific dataset. WE WANT TO AVOID THIS as much as possible.

final preview of what we're generating at the end.


In [200]:

# time reformatting function
def changeTime(df):
    
    for i, row in df.iterrows():
        if 'Q' in row['Dimension_Value_1']:   # it's a quarter
            row['Dimension_Name_1'] = 'Quarter'
            row['Dimension_Value_1'] = row['Dimension_Value_1'].replace('-', '.')
        
        # TODO - NEED MORE SDMX TIME EXAMPLES
    
    # just in case
    assert len(v3_file['Dimension_Name_1'].unique()) == 1
    
    return df


# geography reformatting function
def changeGeog(df):
    
    # TODO - this will be horrible. We're gonna need a whole SMDX pllain text > ONS geographic hierarchy mapping or
    # text matching of some kind. if we're very lucky someone else has already done it.
    
    # If there's only one level its a national code of some kind
    if len(df['Dimension_Value_2'].unique()) == 1:
        oneCode = df['Dimension_Value_2'].unique()[0]
        
        natCodes = {
            "United Kingdom":"K02000001",
            "Great Britain":"K03000001",
            "England and Wales":"K04000001",
            "England":"E92000001",
            "Wales / Cymru":"W92000004",
            "Wales":"W92000004",
            "Cymru":"W92000004",
            "Northern Ireland":"N92000002",
            "Scotland":"S92000003",
                }
        
        # try catch in case we dont know it
        try:
            oneCode = natCodes[oneCode]
        except:
            raise ValueError
        
        df['Dimension_Value_2'] = oneCode
        df['Dimension_Hierarchy_2'] = '2011STATH'
    
    return df
    

v3_file = changeTime(v3_file)
v3_file = changeGeog(v3_file)

v3_file[:10]


,Observation,Data_Marking,Observation_Type_Value,Dimension_Hierarchy_1,Dimension_Name_1,Dimension_Value_1,Dimension_Hierarchy_2,Dimension_Name_2,Dimension_Value_2,Dimension_Hierarchy_3,...,Dimension_Value_5,Dimension_Hierarchy_6,Dimension_Name_6,Dimension_Value_6,Dimension_Hierarchy_7,Dimension_Name_7,Dimension_Value_7,Dimension_Hierarchy_8,Dimension_Name_8,Dimension_Value_8
0,279.902,,,time,Quarter,1995.Q1,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
1,303.099,,,time,Quarter,1995.Q2,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
2,307.714,,,time,Quarter,1995.Q3,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
3,285.639,,,time,Quarter,1995.Q4,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
4,258.886,,,time,Quarter,1996.Q1,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
5,291.308,,,time,Quarter,1996.Q2,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
6,302.456,,,time,Quarter,1996.Q3,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
7,277.518,,,time,Quarter,1996.Q4,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
8,255.679,,,time,Quarter,1997.Q1,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
9,291.108,,,time,Quarter,1997.Q2,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP


In [201]:
# some of the headers are a bit rubbish

v3_file['Dimension_Name_3'] = 'NACE'

v3_file[:10]


,Observation,Data_Marking,Observation_Type_Value,Dimension_Hierarchy_1,Dimension_Name_1,Dimension_Value_1,Dimension_Hierarchy_2,Dimension_Name_2,Dimension_Value_2,Dimension_Hierarchy_3,...,Dimension_Value_5,Dimension_Hierarchy_6,Dimension_Name_6,Dimension_Value_6,Dimension_Hierarchy_7,Dimension_Name_7,Dimension_Value_7,Dimension_Hierarchy_8,Dimension_Name_8,Dimension_Value_8
0,279.902,,,time,Quarter,1995.Q1,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
1,303.099,,,time,Quarter,1995.Q2,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
2,307.714,,,time,Quarter,1995.Q3,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
3,285.639,,,time,Quarter,1995.Q4,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
4,258.886,,,time,Quarter,1996.Q1,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
5,291.308,,,time,Quarter,1996.Q2,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
6,302.456,,,time,Quarter,1996.Q3,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
7,277.518,,,time,Quarter,1996.Q4,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
8,255.679,,,time,Quarter,1997.Q1,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP
9,291.108,,,time,Quarter,1997.Q2,2011STATH,Geography,K02000001,,...,W2,,counterpart_sector,S1,,ref_sector,S1,,sto,EMP


In [202]:
v3_file.to_csv('sample.csv', index=False)